In [ ]:
import matplotlib as plt

In [2]:
import torch
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms

ModuleNotFoundError: No module named 'torch'

In [ ]:
# Setup and Data downloading
data_root = './data'

# Tranform operation to covern PIL to tensor and normalize it
transform = transforms.Compose([transforms.toTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# train=True
train_dataset = torchvision.datasets.CIFAR10(data_room, train=True, download=True, transform=transform)
# train=False
test_dataset = torchvision.datasets.CIFAR10(data_room, train=False, download=True, transform=transform)

In [ ]:
# Data Loading

batch_size = 4

# shuffle=True is important for training, whatevs for testign
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# to visualize data, you are gonna wanna unnormalize it first

In [ ]:
def unnormalize(img):
    img = img / 2 + 0.5
    img = img.numpy()
    return np.transpose(img, (1, 2, 0))

# Define model

In [ ]:
import torch.nn

# output of conv = (w - k + 2*padding) / s + 1

In [ ]:
class Net(nn.Module): # inherit nn.Module
    def __init__(self, num_classes=10):
        self.conv_layer1 = nn.conv2d(3, 6, 5) #last param mean 5x5 window for kernel
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2,2)
        self.conv_layer2 = nn.conv2d(6, 16, 5)
        
        self.fc_layer1 = nn.Linear(16*5*5, 120) #first param is num outputs times the number of window params from prev layer
        self.fc_layer2 = nn.Linear(120, 84)        
        self.fc_layer3 = nn.Linear(84, num_classes) 
        
    def forward(self, x):
        x = self.pool(self.relu(self.conv_layer1(x)))
        x = self.pool(self.relu(self.conv_layer2(x)))
        
        #we do this because the input of the fully connected layer needs to be flattened
        x = x.view(-1, 16*5*5) # or x.view(x.shape[0]), [4, 16, 5, 5] --> 
        x = self.relu(self.fc_layer1(x))
        x = self.relu(self.fc_layer2(x)) 
        x = self.fc_layer3(x) #don't ReLU the last outputs
        return x

# Training

In [ ]:
import torch.optim as optim

In [ ]:
num_epochs = 5
learning_rate = 1e-3  #very important, if you learning rate is too high, your loss values will become very high

net = Net() #instantiate the net we just made

# if you have a GPU
# net.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate) #popular ones are Adam and SGD.
# if you only want to optimize the params of a specific layer
# optimizer = optim.Adam(net.fc_layer3.parameters(), lr = learning_rate)

In [ ]:
for epoch in range(num_epoch):
    total_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        optimizer.zero_grad() # zero out the optimizer
        
        #get data from datalaoder
        images, labels = data
        
        #if you have GPU, .cuda() images and labels
        
        #pass data through model
        out = net(images)
        loss = criterion(out, labels)
        loss.backward()
        # print(loss.grad()) will actally give you the gradients but we dont need this
        optimizer.step()
        
        
        #calculate loss
        
        
        total_loss += loss.item()
        if i % 2000 == 0
            print('Epoch: {:d}/{:d} \t iter: {:d} \t loss: {:.4f}'.format(epoch, num_epochs, i, total_loss/2000)
        
        #update parameters

In [ ]:
save_path = './checkpoints/model.pth'
torch.save(net.state_dict(), save_path)

# Testing

In [ ]:
net = Net()

checkpoint = torch.load(save_path)
net.load_state_dict(checkpoint)

In [ ]:
for i, data in enumerate(test_loader, 0):
    images, labels = data
    
    out = net(images)
    _, predicted = torch.max(out, 1)
    
    print('Predicted: {} \t ground truth: {}'.format(predicted, labels))